# Projeto 2 - Ciência dos Dados

Nome: Ellen Beatriz Shen

Nome: Gabriela Moreno Boriero

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [2]:
import re 

def cleanup(text):
    #Função de limpeza muito simples que troca alguns sinais básicos por espaços
    import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [3]:
gosta_de_acai = cleanup((open("gosta.txt", "r", encoding="utf8").read()).lower())
indiferente_acai = cleanup((open("indiferente.txt", "r", encoding="utf8").read()).lower())
nao_gosta_de_acai = cleanup((open("nao_gosta.txt", "r", encoding="utf8").read()).lower())

In [4]:
tabela_gosta = (pd.Series(gosta_de_acai.split())).value_counts()
tabela_indiferente = (pd.Series(indiferente_acai.split())).value_counts()
tabela_nao_gosta = (pd.Series(nao_gosta_de_acai.split())).value_counts()

In [5]:
tabela_gosta_acai_relativa = (pd.Series(gosta_de_acai.split())).value_counts(True)
tabela_indiferente_acai_relativa = (pd.Series(indiferente_acai.split())).value_counts(True)
tabela_nao_gosta_acai_relativa = (pd.Series(nao_gosta_de_acai.split())).value_counts(True)

In [12]:
gosta_inter = (set(gosta_de_acai.split())).intersection(set(indiferente_acai.split()))
nao_gosta_inter = (set(nao_gosta_de_acai.split())).intersection(set(indiferente_acai.split()))

In [19]:
gosta_total_r = (pd.Series(list(gosta_inter)).value_counts(True))
nao_gosta_total_r = (pd.Series(list(nao_gosta_inter)).value_counts(True))
gosta_total_r

por            0.003322
meu            0.003322
hj             0.003322
promo          0.003322
pai            0.003322
quase          0.003322
ve             0.003322
litros         0.003322
mim            0.003322
tomar          0.003322
rolê           0.003322
gostoso        0.003322
tenho          0.003322
caldo          0.003322
às             0.003322
milkshake      0.003322
debochada      0.003322
ia             0.003322
burguer        0.003322
pedir          0.003322
carne          0.003322
bem            0.003322
kkkkk          0.003322
pastel         0.003322
https          0.003322
lá             0.003322
mal            0.003322
calor          0.003322
cmg            0.003322
da             0.003322
                 ...   
salgados       0.003322
tu             0.003322
fui            0.003322
que            0.003322
pedindo        0.003322
ano            0.003322
um             0.003322
enquanto       0.003322
porém          0.003322
para           0.003322
ata            0

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**